In [2]:
import scanpy as sc

In [5]:
adata = sc.read_h5ad("neonatal_human_normal_per_cell_log_trans.h5ad")

In [8]:
adata.var["name"] = adata.var.index

In [9]:
adata.var["name_splited"] = [elm.split(".")[0] for elm in adata.var.name.tolist()]

In [38]:
# Save to a .txt file
with open("genes_in_the_dataset_actual.txt", "w") as file:
    for item in adata.var.name.to_list():
        file.write("%s , " % item)

In [29]:
import pandas as pd

In [31]:
genes = pd.read_table("mart_homogoly_and_gene_names.txt")

In [54]:
genes = genes[~genes["Gene name"].duplicated()]

In [56]:
adata_var = adata.var.merge(genes, left_on="name", right_on="Gene name", how = "left")

In [57]:
adata_var.index = adata_var["name_splited"]

In [58]:
adata.var = adata_var

In [65]:
adata = adata[:,~adata.var["Gene stable ID"].isna()]

In [69]:
adata = adata[:,adata.var["Mouse homology type"] == "ortholog_one2one"]

In [74]:
adata.var = pd.DataFrame(index = adata.var["Mouse gene stable ID"])

In [81]:
adata.obs = adata.obs[['disease', 'dataset', 'organism', 'cell_type']]

In [82]:
delattr(adata, "uns")

In [85]:
adata.write_h5ad("neonatal_human_normal_per_cell_log_trans_orth.h5ad")

#### OLD

In [ ]:
orth = pd.read_table("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/mouse_human_homology.tsv")

In [ ]:
duplicates = orth[orth.duplicated(subset="Human gene name", keep=False)]

In [ ]:
unique_ids_per_group = duplicates.groupby("Human gene name")["Gene stable ID"].nunique()
groups_with_multiple_ids = unique_ids_per_group[unique_ids_per_group > 1]
genes_with_multiple_ids = unique_ids_per_group[unique_ids_per_group > 1].index.tolist()

In [ ]:
filtered_orth = orth[~orth["Human gene name"].isin(genes_with_multiple_ids)]

In [ ]:
orth_deduplicated = filtered_orth.drop_duplicates(subset="Human gene name", keep="first")

In [ ]:
merged = adata.var.merge(right=orth_deduplicated, left_on="name_splited", right_on="Human gene name", how = "left")

In [ ]:
merged.index = merged["Gene stable ID"].rename("EID")

In [ ]:
adata.var = merged

In [ ]:
adata = adata[:, ~adata.var["Gene stable ID"].isna()]

In [ ]:
adata = adata[:, adata.var["Human homology type"] =="ortholog_one2one"]

In [ ]:
adata.write_h5ad("/d/hpc/projects/FRI/DL/mo6643/MSC/data/data_update_slack/data_splits/data_splits_train_merge/train_data/adata_human_neonatal_orth.h5ad")